# Scoring ProteinGym Data with Prime (Homo)


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
from pathlib import Path
from scipy.stats import spearmanr

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

In [ ]:
model_path = "AI4Protein/Prime_690M_HomoTune"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model.eval()
model = model.to(device)

In [ ]:
@torch.no_grad()
def score(fasta, mutant):
    df = pd.read_csv(mutant)
    sequence = read_seq(fasta)
    tokenied_results = tokenizer(sequence, return_tensors="pt")
    input_ids = tokenied_results.input_ids.to(device)
    attention_mask = tokenied_results.attention_mask.to(device)
    logits = model(input_ids, attention_mask=attention_mask).logits[0, 1:-1, :].log_softmax(dim=-1)
    scores = []
    for mutant in df["mutant"]:
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df["predict_score"] = scores
    return df

In [ ]:
sequence_folder = Path("../proteingym_v1.0_fasta/fasta")
mutant_folder = Path("../proteingym_v1.0_fasta/mutant")
output_folder = Path("../proteingym_v1.0_fasta/scores")

In [ ]:
for file in sequence_folder.glob("*.fasta"):
    stem = file.stem
    df = score(file, mutant_folder / f"{stem}.csv")
    df.to_csv(output_folder / f"{stem}.csv", index=False)
    print(f"Scoring {stem}, rs = {spearmanr(df['score'], df['predict_score']).correlation:.4f}, saved to {output_folder / f'{stem}.csv'}")